# Run TPC-DS power test with gazelle

## Define spark configuration to enable gazelle

In [ ]:
%%init_spark

launcher.conf.set("spark.driver.extraClassPath", "/home/cloudtik/runtime/benchmark-tools/spark-sql-perf/target/scala-2.12/spark-sql-perf_2.12-0.5.1-SNAPSHOT.jar:/home/cloudtik/runtime/hadoop/share/hadoop/tools/lib/aws-java-sdk-bundle-1.11.375.jar:/home/cloudtik/runtime/hadoop/share/hadoop/tools/lib/hadoop-aws-3.2.0.jar:/home/cloudtik/runtime/oap/oap_jars/gazelle-plugin-1.4.0-spark-3.2.1.jar") 
launcher.conf.set("spark.executor.extraClassPath", "/home/cloudtik/runtime/benchmark-tools/spark-sql-perf/target/scala-2.12/spark-sql-perf_2.12-0.5.1-SNAPSHOT.jar:/home/cloudtik/runtime/hadoop/share/hadoop/tools/lib/aws-java-sdk-bundle-1.11.375.jar:/home/cloudtik/runtime/hadoop/share/hadoop/tools/lib/hadoop-aws-3.2.0.jar:/home/cloudtik/runtime/oap/oap_jars/gazelle-plugin-1.4.0-spark-3.2.1.jar") 
launcher.conf.set("spark.sql.broadcastTimeout", "10000") 
launcher.conf.set("spark.executor.instances", "48") 
launcher.conf.set("spark.driver.memory", "20g") 
launcher.conf.set("spark.driver.maxResultSize", "20g") 
launcher.conf.set("spark.executor.cores", "8") 
launcher.conf.set("spark.executor.memory", "8g") 
launcher.conf.set("spark.executor.memoryOverhead", "384") 
launcher.conf.set("spark.memory.offHeap.enabled", "true") 
launcher.conf.set("spark.memory.offHeap.size", "16g") 
launcher.conf.set("spark.dynamicAllocation.enabled", "false") 
launcher.conf.set("spark.executorEnv.CC", "/home/cloudtik/runtime/oap/bin/x86_64-conda_cos6-linux-gnu-cc") 
launcher.conf.set("spark.plugins", "com.intel.oap.GazellePlugin") 
launcher.conf.set("spark.executorEnv.LD_LIBRARY_PATH", "/home/cloudtik/runtime/oap/lib/") 
launcher.conf.set("spark.executorEnv.LIBARROW_DIR", "/home/cloudtik/runtime/oap/") 
launcher.conf.set("spark.shuffle.manager", "org.apache.spark.shuffle.sort.ColumnarShuffleManager") 
launcher.conf.set("spark.sql.join.preferSortMergeJoin", "false ") 
launcher.conf.set("spark.sql.inMemoryColumnarStorage.batchSize", "20480 ") 
launcher.conf.set("spark.sql.execution.arrow.maxRecordsPerBatc", "20480 ") 
launcher.conf.set("spark.sql.parquet.columnarReaderBatchSize", "20480 ") 
launcher.conf.set("spark.sql.autoBroadcastJoinThreshold", "10M") 
launcher.conf.set("spark.sql.broadcastTimeout", "3000") 
launcher.conf.set("spark.sql.crossJoin.enabled", "true") 
launcher.conf.set("spark.sql.columnar.window", "true") 
launcher.conf.set("spark.sql.columnar.sort", "true") 
launcher.conf.set("spark.sql.codegen.wholeStage", "true") 
launcher.conf.set("spark.sql.columnar.codegen.hashAggregate", "false") 
launcher.conf.set("spark.sql.shuffle.partitions", "384") 
launcher.conf.set("spark.kryoserializer.buffer.max", "128m") 
launcher.conf.set("spark.kryoserializer.buffer", "32m") 
launcher.conf.set("spark.oap.sql.columnar.preferColumnar", "false") 
launcher.conf.set("spark.oap.sql.columnar.sortmergejoin.lazyread", "true") 
launcher.conf.set("spark.oap.sql.columnar.sortmergejoin", "true") 
launcher.conf.set("spark.oap.sql.columnar.coreRange", "0-31,64-95|32-63,96-127") 
launcher.conf.set("spark.oap.sql.columnar.joinOptimizationLevel", "18") 
launcher.conf.set("spark.oap.sql.columnar.shuffle.customizedCompression.codec", "lz4") 
launcher.conf.set("spark.executorEnv.ARROW_ENABLE_NULL_CHECK_FOR_GET", "false") 
launcher.conf.set("spark.executorEnv.ARROW_ENABLE_UNSAFE_MEMORY_ACCESS", "true") 

## Define the benchmark configuration

In [ ]:
val scaleFactor = "1000"           // data scale 1GB
val iterations = 1              // how many times to run the whole set of queries.
val format = "parquet"          // support parquer or orc
// support s3a://s3_bucket, gs://gs_bucket, hdfs://namenode_ip:9000
// wasbs://container@storage_account.blob.core.windows.net
// abfs://container@storage_account.dfs.core.windows.net
val fsdir = "hdfs://namenode_ip:9000" 
val partitionTables = true      // create partition tables
val query_filter = Seq()        // Seq() == all queries
// val query_filter = Seq("q1-v2.4", "q2-v2.4") // run subset of queries
val randomizeQueries = false    // run queries in a random order. Recommended for parallel runs.


## Create tables

In [ ]:
// detailed results will be written as JSON to this location.
var resultLocation = s"${fsdir}/shared/data/results/tpcds_${format}/${scaleFactor}/"
var databaseName = s"tpcds_${format}_scale_${scaleFactor}_db"
val use_arrow = true            // when you want to use gazella_plugin to run TPC-DS, you need to set it true.
val data_path = s"${fsdir}/shared/data/tpcds/tpcds_${format}/${scaleFactor}"

if (use_arrow){
    val data_path= s"${fsdir}/shared/data/tpcds/tpcds_${format}/${scaleFactor}"
    resultLocation = s"${fsdir}/shared/data/results/tpcds_arrow/${scaleFactor}/"
    databaseName = s"tpcds_arrow_scale_${scaleFactor}_db"
    val tables = Seq("call_center", "catalog_page", "catalog_returns", "catalog_sales", "customer", "customer_address", "customer_demographics", "date_dim", "household_demographics", "income_band", "inventory", "item", "promotion", "reason", "ship_mode", "store", "store_returns", "store_sales", "time_dim", "warehouse", "web_page", "web_returns", "web_sales", "web_site")
    if (spark.catalog.databaseExists(s"$databaseName")) {
        println(s"$databaseName has exists!")
    }else{
        spark.sql(s"create database if not exists $databaseName").show
        spark.sql(s"use $databaseName").show
        for (table <- tables) {
            if (spark.catalog.tableExists(s"$table")){
                println(s"$table has exists!")
            }else{
                spark.catalog.createTable(s"$table", s"$data_path/$table", "arrow")
            }
        }
        if (partitionTables) {
            for (table <- tables) {
                try{
                    spark.sql(s"ALTER TABLE $table RECOVER PARTITIONS").show
                }catch{
                        case e: Exception => println(e)
                }
            }
        }
    }
}else{
    if (spark.catalog.databaseExists(s"$databaseName")) {
        println(s"$databaseName has exists!")
    }else{
        import com.databricks.spark.sql.perf.tpcds.TPCDSTables

        println(s"$databaseName doesn't exist. Creating...")
        val tables = new TPCDSTables(spark.sqlContext, "", s"${scaleFactor}", false)
        tables.createExternalTables(data_path, format, databaseName, overwrite = true, discoverPartitions = partitionTables)
    }
}

val timeout = 60 // timeout in hours


## Run queries

In [ ]:
sql(s"use $databaseName")
import com.databricks.spark.sql.perf.tpcds.TPCDS
val tpcds = new TPCDS (sqlContext = spark.sqlContext)
def queries = {
  val filtered_queries = query_filter match {
    case Seq() => tpcds.tpcds2_4Queries
    case _ => tpcds.tpcds2_4Queries.filter(q => query_filter.contains(q.name))
  }
  if (randomizeQueries) scala.util.Random.shuffle(filtered_queries) else filtered_queries
}
val experiment = tpcds.runExperiment(
  queries,
  iterations = iterations,
  resultLocation = resultLocation,
  tags = Map("runtype" -> "benchmark", "database" -> databaseName, "scale_factor" -> scaleFactor))

println(experiment.toString)
experiment.waitForFinish(timeout*60*60)
